In [1]:
import pandas as pd 
import numpy as np 

clarity_data = pd.read_csv("cleaned_clarity_data.csv")
clarity_data

,Code,Demand
0,0,2
1,11,1859
2,12,2620
3,13,1156
4,14,131
...,...,...
122,966,5
123,981,1
124,986,1276
125,990,22


In [ ]:
# create demand_prob column using Demand column over all demand
clarity_data['demand_prob'] = clarity_data['Demand'] / clarity_data['Demand'].sum()
clarity_data

,Code,Demand,demand_prob
0,0,2,0.000050
1,11,1859,0.046648
2,12,2620,0.065743
3,13,1156,0.029007
4,14,131,0.003287
...,...,...,...
122,966,5,0.000125
123,981,1,0.000025
124,986,1276,0.032018
125,990,22,0.000552


In [4]:
clarity_data.to_csv("clarity_data_with_demand_prob.csv", index=False)

In [5]:
print(clarity_data)

     Code  Demand  demand_prob
0       0       2     0.000050
1      11    1859     0.046648
2      12    2620     0.065743
3      13    1156     0.029007
4      14     131     0.003287
..    ...     ...          ...
122   966       5     0.000125
123   981       1     0.000025
124   986    1276     0.032018
125   990      22     0.000552
126   999     885     0.022207

[127 rows x 3 columns]


In [ ]:
# Random select of row based on demand_prob, and return the Code column value
def random_select():
    return clarity_data.sample(1, weights=clarity_data['demand_prob'])['Code'].values[0]

# Random select 10 times
for i in range(10):
    print(random_select())
    

986
613
22
574
121
943
121
927
114
22


In [6]:
import pandas as pd

item_code_df = pd.read_csv("cleaned_item_code.csv")

# Define the function
def generate_revenue(clarity_data, item_code_df, total_revenue):
    revenue = 0
    selected_codes = []  # Store selected codes for tracking
    selected_rows = []   # Store the detailed selected rows for output

    while revenue < total_revenue:
        # Randomly select a Code based on demand_prob
        selected_code = clarity_data.sample(1, weights=clarity_data['demand_prob'])['Code'].values[0]
        # Look up the price AUD for the selected Code
        matched_item = item_code_df[item_code_df['item_number'].astype(str).str.zfill(3) == str(selected_code).zfill(3)]
        if not matched_item.empty:
            price_aud = matched_item['price AUD'].values[0]
            # Add the price to revenue and store the selected item
            revenue += price_aud
            selected_codes.append(selected_code)
            selected_rows.append({
                'Code': selected_code,
                'Price AUD': price_aud,
                'Cumulative Revenue': revenue
            })
        else:
            # If no matching code is found in item_code_df, skip
            continue

    # Convert the selected rows into a DataFrame
    result_df = pd.DataFrame(selected_rows)
    return result_df



total_revenue = 300000
result = generate_revenue(clarity_data, item_code_df, total_revenue)
print(result)


      Code  Price AUD  Cumulative Revenue
0      311       89.0                89.0
1       12       33.0               122.0
2      113       86.0               208.0
3       71       33.0               241.0
4      114       83.0               324.0
...    ...        ...                 ...
4031    12       33.0            299607.0
4032    22        6.0            299613.0
4033   986       71.0            299684.0
4034   119      197.0            299881.0
4035   533      186.0            300067.0

[4036 rows x 3 columns]


In [8]:
def generate_revenue_from_clinic_df(clarity_data, item_code_df, clinic_df):
    result_dict = {}  # To store results for each clinic

    for _, row in clinic_df.iterrows():
        clinic_name = row['clinic_name']
        clinic_revenue = row['clinic_revenue']

        revenue = 0
        selected_rows = []  # Store the detailed selected rows for output

        while revenue < clinic_revenue:
            # Randomly select a Code based on demand_prob
            selected_code = clarity_data.sample(1, weights=clarity_data['demand_prob'])['Code'].values[0]
            # Look up the price AUD for the selected Code
            matched_item = item_code_df[item_code_df['item_number'].astype(str).str.zfill(3) == str(selected_code).zfill(3)]
            if not matched_item.empty:
                price_aud = matched_item['price AUD'].values[0]
                # Add the price to revenue and store the selected item
                revenue += price_aud
                selected_rows.append({
                    'Code': selected_code,
                    'Price AUD': price_aud,
                    'Cumulative Revenue': revenue
                })
            else:
                # If no matching code is found in item_code_df, skip
                continue

        # Convert the selected rows into a DataFrame and add to the result_dict
        result_df = pd.DataFrame(selected_rows)
        result_dict[clinic_name] = result_df

    return result_dict

# Example usage
clarity_data = pd.DataFrame({
    'Code': [0, 11, 12, 13, 14],
    'Demand': [2, 1859, 2620, 1156, 131],
    'demand_prob': [0.000050, 0.046648, 0.065743, 0.029007, 0.003287]
})

item_code_df = pd.DataFrame({
    'item_number': ['011', '012', '013', '014', '015'],
    'price AUD': [49.0, 33.0, 30.0, 34.0, 83.0]
})

clinic_df = pd.DataFrame({
    'clinic_name': ['Clinic A'],
    'clinic_revenue': [100000]
})

result_dict = generate_revenue_from_clinic_df(clarity_data, item_code_df, clinic_df)

# Display results for each clinic
for clinic, df in result_dict.items():
    print(f"\n{clinic} Revenue Breakdown:")
    print(df)



Clinic A Revenue Breakdown:
      Code  Price AUD  Cumulative Revenue
0       12       33.0                33.0
1       12       33.0                66.0
2       13       30.0                96.0
3       11       49.0               145.0
4       12       33.0               178.0
...    ...        ...                 ...
2651    11       49.0             99839.0
2652    12       33.0             99872.0
2653    11       49.0             99921.0
2654    12       33.0             99954.0
2655    11       49.0            100003.0

[2656 rows x 3 columns]


In [10]:
result_dict['Clinic A']

,Code,Price AUD,Cumulative Revenue
0,12,33.0,33.0
1,12,33.0,66.0
2,13,30.0,96.0
3,11,49.0,145.0
4,12,33.0,178.0
...,...,...,...
2651,11,49.0,99839.0
2652,12,33.0,99872.0
2653,11,49.0,99921.0
2654,12,33.0,99954.0


### Preprocess Salary (Get the Parameter Distribution)

In [11]:
import numpy as np
import pandas as pd


In [20]:
salary = pd.read_csv("salary_data.csv")
salary

,OHT,Dentist,Specialist
0,35.0,75.0,125
1,34.0,72.0,120
2,32.0,70.0,150
3,36.0,77.0,100
4,33.0,71.0,115
5,32.0,69.0,120
6,35.0,75.0,115
7,34.0,72.0,140
8,43.0,100.0,125
9,42.0,100.0,120


In [21]:
#import norm from scipy.stats
from scipy.stats import norm


# fit a normal distribution to the data of salary['OHT'] 
salary = salary.dropna(subset=['OHT'])
mu, std = norm.fit(salary['OHT'])
mu, std


(41.458333333333336, 6.298010047273309)

In [23]:
salary.columns

Index(['OHT', 'Dentist ', 'Specialist'], dtype='object')

In [24]:
salary = salary.dropna(subset=['Dentist '])
mu, std = norm.fit(salary['Dentist '])
mu, std

(97.83333333333333, 20.5784407140634)

In [25]:
mu, std = norm.fit(salary['Specialist'])
mu, std

(140.33333333333334, 25.142372379886684)

In [30]:
reference_clinic = pd.read_csv("reference_clinic_df_real.csv")
len(reference_clinic[:50])

50

### FROM ITEM CODE TO CASHFLOW

In [3]:
import pandas as pd
import numpy as np
import os
import sys


# Use the current working directory as the base path
sys.path.append(os.path.dirname(os.getcwd()))

# Import model_forecasting.py from the main folder
from model_forecasting import ModelForecastPerformance
# Import model_forecasting.py from the main folder
from model_forecasting import ModelForecastPerformance


clinic_item_code  = pd.read_csv('clinic_item_code.csv')
cleaned_item_code  = pd.read_csv('cleaned_item_code.csv')

# model = ModelForecastPerformance()
# model.forecast_revenue_expenses_with_cov(clinic_item_code, cleaned_item_code, 12, 0.1)


In [3]:
import pandas as pd
import numpy as np
import random
from datetime import date, datetime
import calendar

def generate_date_from_month(month, method='random'):
    """
    Generate a specific date based on a month number and the desired method ('random', 'last_day', or 'first_day').
    """
    start_year = 2024
    year = start_year + (month - 1) // 12
    month = (month - 1) % 12 + 1
    days_in_month = calendar.monthrange(year, month)[1]

    if method == 'random':
        day = random.randint(1, days_in_month)
    elif method == 'last_day':
        day = days_in_month
    elif method == 'first_day':
        day = 1
    else:
        raise ValueError("Invalid method. Choose 'random', 'last_day', or 'first_day'.")

    return date(year, month, day)

def forecast_revenue_expenses_with_cov_optimized_v2(clinic_item_code, cleaned_item_code, forecast_period, target_cov, tolerance=0.1):
    """
    Optimized generation of a cashflow DataFrame with random sampling of revenue and expense,
    ensuring CoV of monthly profits matches the target CoV.
    """
    # Ensure consistent formatting by zero-padding 'Code' to match 'item_number'
    clinic_item_code['Code'] = clinic_item_code['Code'].apply(lambda x: str(x).zfill(3))
    cleaned_item_code['item_number'] = cleaned_item_code['item_number'].astype(str).str.zfill(3)

    # Merge clinic_item_code with cleaned_item_code based on Code and item_number
    merged_df = pd.merge(
        clinic_item_code,
        cleaned_item_code,
        left_on="Code",
        right_on="item_number",
        how="inner"
    )
    
    # Extract necessary columns
    merged_df = merged_df[['Code', 'Total Demand', 'price AUD', 'cost_material AUD']]
    merged_df['Demand per Period'] = (merged_df['Total Demand'] / 12) * forecast_period

    # Initialize storage for allocations
    allocations = []

    # Allocate demand for each Code
    for _, row in merged_df.iterrows():
        remaining_demand = row['Demand per Period']
        price = row['price AUD']
        cost = row['cost_material AUD']
        
        while remaining_demand > 0:
            # Generate a random date within the forecast period
            random_month = random.randint(1, forecast_period)
            generated_date = generate_date_from_month(random_month)

            # Allocate a random fraction of the remaining demand
            allocated_demand = min(remaining_demand, np.random.uniform(0, remaining_demand))
            remaining_demand -= allocated_demand

            # Collect the allocation
            allocations.append({
                'Date': generated_date,
                'Code': row['Code'],
                'Revenue': allocated_demand * price,
                'Expense': allocated_demand * cost
            })

    # Convert allocations to a DataFrame
    allocation_df = pd.DataFrame(allocations)

    # Aggregate to monthly totals
    allocation_df['Month'] = allocation_df['Date'].apply(lambda d: d.strftime('%Y-%m'))
    monthly_agg = allocation_df.groupby('Month').agg(
        Revenue=('Revenue', 'sum'),
        Expense=('Expense', 'sum')
    ).reset_index()

    # Calculate monthly profit
    monthly_agg['Profit'] = monthly_agg['Revenue'] - monthly_agg['Expense']

    # Calculate CoV of monthly profit
    def calculate_cov(values):
        mean = np.mean(values)
        std_dev = np.std(values)
        return std_dev / mean if mean != 0 else 0

    current_cov = calculate_cov(monthly_agg['Profit'])

    # Adjust CoV if necessary, batch process
    max_iterations = 20
    iteration = 0

    while not (target_cov * (target_cov-tolerance) <= current_cov <= target_cov * (target_cov+tolerance)) and iteration < max_iterations:
        # Identify months with extreme deviations
        profit_mean = monthly_agg['Profit'].mean()
        extreme_months = monthly_agg.loc[
            abs(monthly_agg['Profit'] - profit_mean) > (profit_mean * target_cov), 'Month'
        ].tolist()

        # Reallocate entries for these months
        if extreme_months:
            reallocated = allocation_df[allocation_df['Month'].isin(extreme_months)].copy()
            allocation_df = allocation_df[~allocation_df['Month'].isin(extreme_months)]

            # Redistribute the reallocated entries
            for i in range(len(reallocated)):
                random_month = random.randint(1, forecast_period)
                reallocated.iloc[i, reallocated.columns.get_loc('Date')] = generate_date_from_month(random_month)
                reallocated.iloc[i, reallocated.columns.get_loc('Month')] = reallocated.iloc[i]['Date'].strftime('%Y-%m')

            allocation_df = pd.concat([allocation_df, reallocated], ignore_index=True)

        # Recalculate monthly aggregates and CoV
        monthly_agg = allocation_df.groupby('Month').agg(
            Revenue=('Revenue', 'sum'),
            Expense=('Expense', 'sum')
        ).reset_index()
        monthly_agg['Profit'] = monthly_agg['Revenue'] - monthly_agg['Expense']
        current_cov = calculate_cov(monthly_agg['Profit'])
        iteration += 1

    # Final cashflow DataFrame
    cashflow_df = monthly_agg.copy()
    cashflow_df['Month'] = pd.to_datetime(cashflow_df['Month'], format='%Y-%m')
    cashflow_df = cashflow_df.sort_values(by='Month').reset_index(drop=True)

    return cashflow_df

In [7]:
forecast_revenue_expenses_with_cov_optimized_v2(clinic_item_code, cleaned_item_code, 12, 0.1, 0.5)

KeyboardInterrupt: 

In [4]:
def forecast_revenue_expenses_with_cov_final(clinic_item_code, cleaned_item_code, forecast_period, target_cov):
    """
    Optimized generation of a cashflow DataFrame with random sampling of revenue and expense,
    ensuring CoV of monthly profits matches the target CoV.
    """
    # Ensure consistent formatting by zero-padding 'Code' to match 'item_number'
    clinic_item_code['Code'] = clinic_item_code['Code'].apply(lambda x: str(x).zfill(3))
    cleaned_item_code['item_number'] = cleaned_item_code['item_number'].astype(str).str.zfill(3)

    # Merge clinic_item_code with cleaned_item_code based on Code and item_number
    merged_df = pd.merge(
        clinic_item_code,
        cleaned_item_code,
        left_on="Code",
        right_on="item_number",
        how="inner"
    )
    
    # Extract necessary columns
    merged_df = merged_df[['Code', 'Total Demand', 'price AUD', 'cost_material AUD']]
    merged_df['Demand per Period'] = (merged_df['Total Demand'] / 12) * forecast_period

    # Initialize storage for allocations
    allocations = []

    # Allocate demand for each Code
    for _, row in merged_df.iterrows():
        remaining_demand = row['Demand per Period']
        price = row['price AUD']
        cost = row['cost_material AUD']
        
        while remaining_demand > 0:
            # Generate a random date within the forecast period
            random_month = random.randint(1, forecast_period)
            generated_date = generate_date_from_month(random_month)

            # Allocate a random fraction of the remaining demand
            allocated_demand = min(remaining_demand, np.random.uniform(0, remaining_demand))
            remaining_demand -= allocated_demand

            # Collect the allocation
            allocations.append({
                'Date': generated_date,
                'Code': row['Code'],
                'Revenue': allocated_demand * price,
                'Expense': allocated_demand * cost
            })

    # Convert allocations to a DataFrame
    allocation_df = pd.DataFrame(allocations)

    # Aggregate to monthly totals
    allocation_df['Month'] = allocation_df['Date'].apply(lambda d: d.strftime('%Y-%m'))
    monthly_agg = allocation_df.groupby('Month').agg(
        Revenue=('Revenue', 'sum'),
        Expense=('Expense', 'sum')
    ).reset_index()

    # Calculate monthly profit
    monthly_agg['Profit'] = monthly_agg['Revenue'] - monthly_agg['Expense']

    # Calculate CoV of monthly profit
    def calculate_cov(values):
        mean = np.mean(values)
        std_dev = np.std(values)
        return std_dev / mean if mean != 0 else 0

    current_cov = calculate_cov(monthly_agg['Profit'])

    # Adjust CoV if necessary, batch process
    max_iterations = 10
    iteration = 0

    while not (target_cov * 0.9 <= current_cov <= target_cov * 1.1) and iteration < max_iterations:
        # Identify months with extreme deviations
        profit_mean = monthly_agg['Profit'].mean()
        extreme_months = monthly_agg.loc[
            abs(monthly_agg['Profit'] - profit_mean) > (profit_mean * target_cov), 'Month'
        ].tolist()

        if not extreme_months:
            break

        # Reallocate entries for these months in batch
        reallocated = allocation_df[allocation_df['Month'].isin(extreme_months)].copy()
        allocation_df = allocation_df[~allocation_df['Month'].isin(extreme_months)]

        # Redistribute the reallocated entries in a vectorized manner
        new_months = [random.randint(1, forecast_period) for _ in range(len(reallocated))]
        reallocated['Date'] = [generate_date_from_month(month) for month in new_months]
        reallocated['Month'] = reallocated['Date'].apply(lambda d: d.strftime('%Y-%m'))

        # Append updated reallocated entries back to the allocation DataFrame
        allocation_df = pd.concat([allocation_df, reallocated], ignore_index=True)

        # Recalculate monthly aggregates and CoV
        monthly_agg = allocation_df.groupby('Month').agg(
            Revenue=('Revenue', 'sum'),
            Expense=('Expense', 'sum')
        ).reset_index()
        monthly_agg['Profit'] = monthly_agg['Revenue'] - monthly_agg['Expense']
        current_cov = calculate_cov(monthly_agg['Profit'])
        iteration += 1

    # Final cashflow DataFrame
    cashflow_df = monthly_agg.copy()
    cashflow_df['Month'] = pd.to_datetime(cashflow_df['Month'], format='%Y-%m')
    cashflow_df = cashflow_df.sort_values(by='Month').reset_index(drop=True)

    return cashflow_df

In [6]:
forecast_revenue_expenses_with_cov_final(clinic_item_code, cleaned_item_code, 12, 0.1)

,Month,Revenue,Expense,Profit
0,2024-01-01,12421.397214,1263.012235,11158.384980
1,2024-02-01,43804.981046,4090.731046,39714.249999
2,2024-03-01,80386.342399,7773.701079,72612.641320
3,2024-04-01,100383.416772,11361.128279,89022.288493
4,2024-05-01,39883.832074,3785.235670,36098.596403
5,2024-06-01,186015.384885,15908.538847,170106.846038
6,2024-07-01,208001.264281,20726.345692,187274.918588
7,2024-08-01,33939.906258,2746.260797,31193.645461
8,2024-09-01,110496.888104,13795.985072,96700.903032
9,2024-10-01,42022.776329,4390.490261,37632.286068


In [7]:
def forecast_revenue_expenses_with_cov_optimized(clinic_item_code, cleaned_item_code, forecast_period, target_cov):
    """
    Optimized generation of a cashflow DataFrame with random sampling of revenue and expense,
    ensuring CoV of monthly profits matches the target CoV, using arrays for efficiency.
    """
    # Ensure consistent formatting by zero-padding 'Code' to match 'item_number'
    clinic_item_code['Code'] = clinic_item_code['Code'].apply(lambda x: str(x).zfill(3))
    cleaned_item_code['item_number'] = cleaned_item_code['item_number'].astype(str).str.zfill(3)

    # Merge clinic_item_code with cleaned_item_code based on Code and item_number
    merged_df = pd.merge(
        clinic_item_code,
        cleaned_item_code,
        left_on="Code",
        right_on="item_number",
        how="inner"
    )
    
    # Extract necessary columns
    merged_df = merged_df[['Code', 'Total Demand', 'price AUD', 'cost_material AUD']]
    merged_df['Demand per Period'] = (merged_df['Total Demand'] / 12) * forecast_period

    # Initialize storage for allocations
    allocations = []

    # Precompute random dates for efficiency
    random_dates = [
        generate_date_from_month(random.randint(1, forecast_period)) for _ in range(100000)
    ]
    date_index = 0

    # Allocate demand for each Code
    for _, row in merged_df.iterrows():
        remaining_demand = row['Demand per Period']
        price = row['price AUD']
        cost = row['cost_material AUD']
        
        while remaining_demand > 0:
            # Use precomputed random date
            generated_date = random_dates[date_index]
            date_index = (date_index + 1) % len(random_dates)

            # Allocate a random fraction of the remaining demand
            allocated_demand = min(remaining_demand, np.random.uniform(0, remaining_demand))
            remaining_demand -= allocated_demand

            # Collect the allocation
            allocations.append([generated_date, row['Code'], allocated_demand * price, allocated_demand * cost])

    # Convert allocations to a NumPy array for efficiency
    allocations = np.array(allocations, dtype=object)

    # Aggregate to monthly totals
    months = np.array([alloc[0].strftime('%Y-%m') for alloc in allocations])
    revenues = allocations[:, 2].astype(float)
    expenses = allocations[:, 3].astype(float)
    unique_months, month_indices = np.unique(months, return_inverse=True)

    monthly_revenue = np.zeros(len(unique_months))
    monthly_expense = np.zeros(len(unique_months))

    np.add.at(monthly_revenue, month_indices, revenues)
    np.add.at(monthly_expense, month_indices, expenses)

    monthly_profit = monthly_revenue - monthly_expense

    # Calculate CoV
    def calculate_cov(values):
        mean = np.mean(values)
        std_dev = np.std(values)
        return std_dev / mean if mean != 0 else 0

    current_cov = calculate_cov(monthly_profit)

    # Adjust CoV if necessary, batch process
    max_iterations = 10
    iteration = 0

    while not (target_cov * 0.9 <= current_cov <= target_cov * 1.1) and iteration < max_iterations:
        # Identify months with extreme deviations
        profit_mean = np.mean(monthly_profit)
        deviations = abs(monthly_profit - profit_mean)
        extreme_indices = np.where(deviations > (profit_mean * target_cov))[0]

        if len(extreme_indices) == 0:
            break

        # Reallocate entries for extreme months in batch
        for idx in extreme_indices:
            affected_rows = np.where(month_indices == idx)[0]
            for row_idx in affected_rows:
                new_month = random.randint(1, forecast_period)
                allocations[row_idx][0] = generate_date_from_month(new_month)

        # Recalculate monthly aggregates and CoV
        months = np.array([alloc[0].strftime('%Y-%m') for alloc in allocations])
        unique_months, month_indices = np.unique(months, return_inverse=True)

        monthly_revenue.fill(0)
        monthly_expense.fill(0)

        np.add.at(monthly_revenue, month_indices, revenues)
        np.add.at(monthly_expense, month_indices, expenses)

        monthly_profit = monthly_revenue - monthly_expense
        current_cov = calculate_cov(monthly_profit)
        iteration += 1

    # Convert results back to a DataFrame
    cashflow_df = pd.DataFrame({
        'Month': pd.to_datetime(unique_months, format='%Y-%m'),
        'Revenue': monthly_revenue,
        'Expense': monthly_expense,
        'Profit': monthly_profit
    }).sort_values(by='Month').reset_index(drop=True)

    return cashflow_df

# Generate optimized cashflow_df with CoV control
cashflow_df_array_optimized = forecast_revenue_expenses_with_cov_optimized(
    clinic_item_code, cleaned_item_code, forecast_period=12, target_cov=0.15
)
cashflow_df_array_optimized


,Month,Revenue,Expense,Profit
0,2024-01-01,154602.188153,17803.049733,136799.138421
1,2024-02-01,36713.657125,3666.404322,33047.252803
2,2024-03-01,59272.017778,4866.761543,54405.256234
3,2024-04-01,166416.926967,13825.587411,152591.339556
4,2024-05-01,196402.343292,21670.568529,174731.774763
5,2024-06-01,135318.989584,16465.351231,118853.638353
6,2024-07-01,75896.321083,6727.658278,69168.662805
7,2024-08-01,192821.791965,15730.304501,177091.487465
8,2024-09-01,40052.037446,3902.969319,36149.068127
9,2024-10-01,16876.088718,1558.087819,15318.000899


In [8]:
# calculate the CoV of the profit
def calculate_cov(values):
    mean = np.mean(values)
    std_dev = np.std(values)
    return std_dev / mean if mean != 0 else 0

# Calculate CoV of the profit
cov_profit = calculate_cov(cashflow_df_array_optimized['Profit'])
cov_profit

0.6545359418250278

In [ ]:
def generate_monthly_cashflow_given_item_code(clinic_item_code, forecast_period, target_cov):
    """
    Optimized generation of a monthly cashflow DataFrame with controlled CoV for profits.
    """
    # Calculate total revenue and expense
    total_revenue = (clinic_item_code['Total Demand'] * clinic_item_code['price AUD']).sum()
    total_expense = (clinic_item_code['Total Demand'] * clinic_item_code['cost_material AUD']).sum()
    total_profit = total_revenue - total_expense

    # Initialize monthly cashflow
    monthly_cashflow = pd.DataFrame({
        'Period': list(range(1, forecast_period + 1)),
        'Revenue': np.zeros(forecast_period),
        'Expense': np.zeros(forecast_period),
        'Profit': np.zeros(forecast_period)
    })

    # Generate Revenue and Expense with controlled CoV for Profit
    mean_revenue = total_revenue / forecast_period
    mean_expense = total_expense / forecast_period
    mean_profit = mean_revenue - mean_expense

    std_profit = target_cov * mean_profit

    # Generate profit distribution with target CoV
    profit_dist = np.random.normal(loc=mean_profit, scale=std_profit, size=forecast_period)
    profit_dist = np.clip(profit_dist, 0, None)  # Ensure no negative profits

    # Adjust Revenue and Expense to match total and profit distribution
    revenue_dist = profit_dist + mean_expense
    expense_dist = revenue_dist - profit_dist

    # Scale Revenue and Expense to match totals
    revenue_dist *= total_revenue / revenue_dist.sum()
    expense_dist *= total_expense / expense_dist.sum()

    # Populate monthly cashflow
    monthly_cashflow['Revenue'] = revenue_dist
    monthly_cashflow['Expense'] = expense_dist
    monthly_cashflow['Profit'] = monthly_cashflow['Revenue'] - monthly_cashflow['Expense']

    return monthly_cashflow

def populate_item_codes_by_period(clinic_item_code, monthly_cashflow):
    """
    Populate item codes into periods based on target profits in the monthly cashflow.
    """
    # Initialize a DataFrame for the final output
    populated_data = []

    # Create a working copy of clinic_item_code with a Remaining Demand column
    clinic_item_code = clinic_item_code.copy()
    clinic_item_code['Remaining Demand'] = clinic_item_code['Total Demand']

    for _, period_row in monthly_cashflow.iterrows():
        period = period_row['Period']
        target_profit = period_row['Profit']
        accumulated_profit = 0

        while accumulated_profit < target_profit and clinic_item_code['Remaining Demand'].sum() > 0:
            # Randomly select an item code
            available_items = clinic_item_code[clinic_item_code['Remaining Demand'] > 0]
            sampled_row = available_items.sample(n=1).iloc[0]

            # Calculate the sampled profit
            sampled_revenue = sampled_row['price AUD']
            sampled_expense = sampled_row['cost_material AUD']
            sampled_profit = sampled_revenue - sampled_expense

            # Deduct one unit of demand from the sampled item code
            clinic_item_code.loc[sampled_row.name, 'Remaining Demand'] -= 1

            # Append to the populated data
            populated_data.append({
                'Period': period,
                'Code': sampled_row['Code'],
                'Revenue': sampled_revenue,
                'Expense': sampled_expense,
                'Profit': sampled_profit
            })

            accumulated_profit += sampled_profit

    # Handle remaining item codes in the last period
    last_period = monthly_cashflow['Period'].max()
    remaining_items = clinic_item_code[clinic_item_code['Remaining Demand'] > 0]

    for _, remaining_row in remaining_items.iterrows():
        while remaining_row['Remaining Demand'] > 0:
            remaining_row['Remaining Demand'] -= 1
            populated_data.append({
                'Period': last_period,
                'Code': remaining_row['Code'],
                'Revenue': remaining_row['price AUD'],
                'Expense': remaining_row['cost_material AUD'],
                'Profit': remaining_row['price AUD'] - remaining_row['cost_material AUD']
            })

    # Convert to DataFrame
    populated_df = pd.DataFrame(populated_data)

    return populated_df


# Step 1: Generate the monthly cashflow with controlled CoV
clinic_item_code['price AUD'] = cleaned_item_code['price AUD']
clinic_item_code['cost_material AUD'] = cleaned_item_code['cost_material AUD']
forecast_period = 12
target_cov = 0.15

monthly_cashflow_df = generate_monthly_cashflow_given_item_code(clinic_item_code, forecast_period, target_cov)

# Step 2: Populate item codes into each period based on the monthly cashflow
populated_item_codes_df = populate_item_codes_by_period(clinic_item_code, monthly_cashflow_df)

In [25]:
populated_item_codes_df 

,Period,Code,Revenue,Expense,Profit
0,1.0,743,400.0,45.59,354.41
1,1.0,019,103.0,13.24,89.76
2,1.0,965,297.0,29.79,267.21
3,1.0,231,0.0,0.00,0.00
4,1.0,627,70.0,6.16,63.84
...,...,...,...,...,...
15431,12.0,022,17.0,1.62,15.38
15432,12.0,022,17.0,1.62,15.38
15433,12.0,022,17.0,1.62,15.38
15434,12.0,022,17.0,1.62,15.38


In [26]:
populated_item_codes_df_group = populated_item_codes_df.groupby('Period').agg(
    Revenue=('Revenue', 'sum'),
    Expense=('Expense', 'sum'),
    Profit=('Profit', 'sum')
).reset_index() 

populated_item_codes_df_group


,Period,Revenue,Expense,Profit
0,1.0,54068.0,5748.97,48319.03
1,2.0,53555.0,5655.49,47899.51
2,3.0,67353.0,7194.38,60158.62
3,4.0,49378.0,5431.32,43946.68
4,5.0,55985.0,5883.66,50101.34
5,6.0,81870.0,8669.47,73200.53
6,7.0,60829.0,6347.01,54481.99
7,8.0,69198.0,7097.95,62100.05
8,9.0,62449.0,6793.63,55655.37
9,10.0,50243.0,5923.44,44319.56


In [27]:
#calculate the CoV of the profit
cov_profit = calculate_cov(populated_item_codes_df_group['Profit'])
cov_profit

0.1544085253749947

In [32]:
grouped_by_code_count = populated_item_codes_df.groupby('Code').size()
grouped_by_code_count

Code
011     697
012    1027
013     417
014      57
015       5
       ... 
966       3
981       1
986     462
990      11
999     320
Length: 102, dtype: int64

In [33]:
clinic_item_code

,Code,Total Demand,Total Revenue,Total Material Cost,Total Duration,Medical Officer,Total Salary,price AUD,cost_material AUD
0,011,697,34153.0,5220.53,20910.0,OHT,13440.904116,49.0,7.49
1,012,1027,33891.0,3799.90,15405.0,OHT,9902.301669,33.0,3.70
2,013,417,12510.0,1617.96,6255.0,OHT,4020.700873,30.0,3.88
3,014,57,1938.0,324.33,855.0,OHT,549.592206,34.0,5.69
4,015,5,415.0,64.75,150.0,OHT,96.419685,83.0,12.95
...,...,...,...,...,...,...,...,...,...
97,966,3,222.0,37.23,45.0,OHT,28.925906,294.0,45.15
98,981,1,125.0,15.40,30.0,OHT,19.283937,86.0,8.93
99,986,462,32802.0,3723.72,13860.0,OHT,8909.178912,297.0,33.28
100,990,11,1474.0,220.77,495.0,OHT,318.184961,86.0,13.46


In [28]:
def convert_period_to_date_with_calendar_logic(monthly_cashflow_df):
    """
    Convert numerical periods into specific dates based on a calendar starting in January 2024.
    The function assigns a random date within the corresponding month.

    Parameters:
    - monthly_cashflow_df: DataFrame with a Period column.

    Returns:
    - DataFrame with an added Date column.
    """
    # Starting year
    start_year = 2024

    # Function to calculate the specific date for a given period
    def random_date_from_period(period):
        # Calculate the year and month based on the period
        year = start_year + (period - 1) // 12
        month = (period - 1) % 12 + 1

        # Get the number of days in the calculated month and year
        days_in_month = calendar.monthrange(year, month)[1]

        # Generate a random day within the month
        random_day = random.randint(1, days_in_month)

        return date(year, month, random_day)

    # Apply the function to the Period column
    monthly_cashflow_df['Period'] = monthly_cashflow_df['Period'].astype(int)
    monthly_cashflow_df['Date'] = monthly_cashflow_df['Period'].apply(random_date_from_period)
    monthly_cashflow_df['Period'] = monthly_cashflow_df['Date']
    
    monthly_cashflow_df.drop(columns=['Date'], inplace=True)

    return monthly_cashflow_df


# Convert Period to actual dates starting from January 2024
converted_cashflow_df = convert_period_to_date_with_calendar_logic(populated_item_codes_df)

In [31]:
converted_cashflow_df.tail(100)

,Period,Code,Revenue,Expense,Profit
15336,2024-12-23,022,17.0,1.62,15.38
15337,2024-12-15,022,17.0,1.62,15.38
15338,2024-12-17,022,17.0,1.62,15.38
15339,2024-12-08,022,17.0,1.62,15.38
15340,2024-12-12,022,17.0,1.62,15.38
...,...,...,...,...,...
15431,2024-12-13,022,17.0,1.62,15.38
15432,2024-12-15,022,17.0,1.62,15.38
15433,2024-12-22,022,17.0,1.62,15.38
15434,2024-12-16,022,17.0,1.62,15.38


In [39]:
cleaned_item_code

,item_number,duration,price AUD,cost_material AUD,price IDR,cost_material IDR,medical_officer,medical_officer_new
0,011,30.0,49.0,7.49,160000,48000,GP,OHT
1,012,15.0,33.0,3.70,140000,42000,GP,OHT
2,013,15.0,30.0,3.88,110000,33000,GP,OHT
3,014,15.0,34.0,5.69,100000,30000,GP,OHT
4,015,30.0,83.0,12.95,260000,78000,GP,OHT
...,...,...,...,...,...,...,...,...
337,986,30.0,71.0,8.06,256500,76950,DNT,OHT
338,987,30.0,67.0,5.33,73500,22050,DNT,OHT
339,990,45.0,134.0,20.07,73500,22050,DNT,OHT
340,999,30.0,0.0,0.00,0,0,-,Specialist


In [40]:
def generate_complete_cashflow(clinic_item_code, cleaned_item_code, forecast_period, target_cov):
    """
    Comprehensive function to generate a complete cashflow DataFrame.
    Includes merging, cashflow generation, item code allocation, and period-to-date conversion.

    Parameters:
    - clinic_item_code: DataFrame with clinic item details including demand and pricing.
    - cleaned_item_code: DataFrame with additional item details for merging.
    - forecast_period: Number of forecast periods (e.g., months).
    - target_cov: Target Coefficient of Variation (CoV) for profit.

    Returns:
    - DataFrame with Period (date), Code, Revenue, Expense, and Profit columns.
    """

    import pandas as pd
    import numpy as np
    import random
    from datetime import date
    import calendar

    clinic_item_code['price AUD'] = cleaned_item_code['price AUD']
    clinic_item_code['cost_material AUD'] = cleaned_item_code['cost_material AUD']

    # Helper: Calculate CoV
    def calculate_cov(values):
        mean = np.mean(values)
        std_dev = np.std(values)
        return std_dev / mean if mean != 0 else 0

    # Step 1: Merge clinic_item_code with cleaned_item_code for necessary details
    clinic_item_code = clinic_item_code.copy()
    clinic_item_code['Code'] = clinic_item_code['Code'].apply(lambda x: str(x).zfill(3))
    cleaned_item_code['item_number'] = cleaned_item_code['item_number'].astype(str).str.zfill(3)
    merged_item_code = pd.merge(
        clinic_item_code,
        cleaned_item_code[['item_number', 'price AUD', 'cost_material AUD']],
        left_on="Code",
        right_on="item_number",
        how="inner"
    )
    # Use only relevant columns
    merged_item_code = merged_item_code[['Code', 'Total Demand', 'price AUD', 'cost_material AUD']]

    # Step 2: Generate monthly cashflow with controlled CoV
    def generate_monthly_cashflow_optimized(merged_item_code, forecast_period, target_cov):
        total_revenue = (merged_item_code['Total Demand'] * merged_item_code['price AUD']).sum()
        total_expense = (merged_item_code['Total Demand'] * merged_item_code['cost_material AUD']).sum()
        mean_revenue = total_revenue / forecast_period
        mean_expense = total_expense / forecast_period
        mean_profit = mean_revenue - mean_expense
        std_profit = target_cov * mean_profit

        # Generate profit distribution
        profit_dist = np.random.normal(loc=mean_profit, scale=std_profit, size=forecast_period)
        profit_dist = np.clip(profit_dist, 0, None)
        revenue_dist = profit_dist + mean_expense
        expense_dist = revenue_dist - profit_dist

        # Scale to match totals
        revenue_dist *= total_revenue / revenue_dist.sum()
        expense_dist *= total_expense / expense_dist.sum()

        return pd.DataFrame({
            'Period': list(range(1, forecast_period + 1)),
            'Revenue': revenue_dist,
            'Expense': expense_dist,
            'Profit': revenue_dist - expense_dist
        })

    monthly_cashflow = generate_monthly_cashflow_optimized(merged_item_code, forecast_period, target_cov)

    # Step 3: Populate item codes by period based on the monthly cashflow
    def populate_item_codes_by_period(merged_item_code, monthly_cashflow):
        clinic_item_code = merged_item_code.copy()
        clinic_item_code['Remaining Demand'] = clinic_item_code['Total Demand']
        populated_data = []

        for _, period_row in monthly_cashflow.iterrows():
            period = period_row['Period']
            target_profit = period_row['Profit']
            accumulated_profit = 0

            while accumulated_profit < target_profit and clinic_item_code['Remaining Demand'].sum() > 0:
                available_items = clinic_item_code[clinic_item_code['Remaining Demand'] > 0]
                sampled_row = available_items.sample(n=1).iloc[0]
                sampled_revenue = sampled_row['price AUD']
                sampled_expense = sampled_row['cost_material AUD']
                sampled_profit = sampled_revenue - sampled_expense

                clinic_item_code.loc[sampled_row.name, 'Remaining Demand'] -= 1
                populated_data.append({
                    'Period': period,
                    'Code': sampled_row['Code'],
                    'Revenue': sampled_revenue,
                    'Expense': sampled_expense,
                    'Profit': sampled_profit
                })

                accumulated_profit += sampled_profit

        # Handle remaining items in the last period
        last_period = monthly_cashflow['Period'].max()
        remaining_items = clinic_item_code[clinic_item_code['Remaining Demand'] > 0]
        for _, remaining_row in remaining_items.iterrows():
            while remaining_row['Remaining Demand'] > 0:
                remaining_row['Remaining Demand'] -= 1
                populated_data.append({
                    'Period': last_period,
                    'Code': remaining_row['Code'],
                    'Revenue': remaining_row['price AUD'],
                    'Expense': remaining_row['cost_material AUD'],
                    'Profit': remaining_row['price AUD'] - remaining_row['cost_material AUD']
                })

        return pd.DataFrame(populated_data)

    populated_item_codes = populate_item_codes_by_period(merged_item_code, monthly_cashflow)

    # Step 4: Convert numerical periods into calendar dates
    def convert_period_to_date_with_calendar_logic(populated_item_codes):
        start_year = 2024

        def random_date_from_period(period):
            year = start_year + (period - 1) // 12
            month = (period - 1) % 12 + 1
            days_in_month = calendar.monthrange(year, month)[1]
            random_day = random.randint(1, days_in_month)
            return date(year, month, random_day)

        populated_item_codes['Period'] = populated_item_codes['Period'].astype(int)
        populated_item_codes['Date'] = populated_item_codes['Period'].apply(random_date_from_period)
        return populated_item_codes

    populated_item_codes = convert_period_to_date_with_calendar_logic(populated_item_codes)

    return populated_item_codes

clinic_item_code['price AUD'] = cleaned_item_code['price AUD']
clinic_item_code['cost_material AUD'] = cleaned_item_code['cost_material AUD']
# Example usage
complete_cashflow_df = generate_complete_cashflow(clinic_item_code, cleaned_item_code, forecast_period=12, target_cov=0.15)
complete_cashflow_df

KeyError: "['price AUD', 'cost_material AUD'] not in index"

In [ ]:
def generate_complete_cashflow_final_debug(clinic_item_code, cleaned_item_code, forecast_period, target_cov):
    """
    Generate a complete cashflow DataFrame with item codes distributed across periods,
    and convert numerical periods into calendar dates.

    Parameters:
    - clinic_item_code: DataFrame with clinic item details including demand and pricing.
    - cleaned_item_code: DataFrame with additional item details for merging.
    - forecast_period: Number of forecast periods (e.g., months).
    - target_cov: Target Coefficient of Variation (CoV) for profit.

    Returns:
    - DataFrame with Period (date), Code, Revenue, Expense, and Profit columns.
    """
    import pandas as pd
    import numpy as np
    import random
    from datetime import date
    import calendar

    # Helper: Calculate CoV
    def calculate_cov(values):
        mean = np.mean(values)
        std_dev = np.std(values)
        return std_dev / mean if mean != 0 else 0

    # Step 1: Merge clinic_item_code with cleaned_item_code for necessary details
    clinic_item_code = clinic_item_code.copy()
    clinic_item_code['Code'] = clinic_item_code['Code'].apply(lambda x: str(x).zfill(3))
    cleaned_item_code['item_number'] = cleaned_item_code['item_number'].astype(str).str.zfill(3)
    merged_item_code = pd.merge(
        clinic_item_code,
        cleaned_item_code[['item_number', 'price AUD', 'cost_material AUD']],
        left_on="Code",
        right_on="item_number",
        how="inner"
    )

    # Verify the column names after merge
    print("Columns in merged_item_code:", merged_item_code.columns)

    # Select correct revenue and expense columns dynamically
    price_col = 'price AUD' if 'price AUD' in merged_item_code.columns else 'price AUD_y'
    cost_col = 'cost_material AUD' if 'cost_material AUD' in merged_item_code.columns else 'cost_material AUD_y'

    # Use only relevant columns
    merged_item_code = merged_item_code[['Code', 'Total Demand', price_col, cost_col]].rename(
        columns={price_col: 'price AUD', cost_col: 'cost_material AUD'}
    )

    # Step 2: Generate monthly cashflow with controlled CoV
    def generate_monthly_cashflow_optimized(merged_item_code, forecast_period, target_cov):
        total_revenue = (merged_item_code['Total Demand'] * merged_item_code['price AUD']).sum()
        total_expense = (merged_item_code['Total Demand'] * merged_item_code['cost_material AUD']).sum()
        mean_revenue = total_revenue / forecast_period
        mean_expense = total_expense / forecast_period
        mean_profit = mean_revenue - mean_expense
        std_profit = target_cov * mean_profit

        # Generate profit distribution
        profit_dist = np.random.normal(loc=mean_profit, scale=std_profit, size=forecast_period)
        profit_dist = np.clip(profit_dist, 0, None)
        revenue_dist = profit_dist + mean_expense
        expense_dist = revenue_dist - profit_dist

        # Scale to match totals
        revenue_dist *= total_revenue / revenue_dist.sum()
        expense_dist *= total_expense / expense_dist.sum()

        return pd.DataFrame({
            'Period': list(range(1, forecast_period + 1)),
            'Revenue': revenue_dist,
            'Expense': expense_dist,
            'Profit': revenue_dist - expense_dist
        })

    monthly_cashflow = generate_monthly_cashflow_optimized(merged_item_code, forecast_period, target_cov)

    # Step 3: Populate item codes by period based on the monthly cashflow
    def populate_item_codes_by_period(merged_item_code, monthly_cashflow):
        clinic_item_code = merged_item_code.copy()
        clinic_item_code['Remaining Demand'] = clinic_item_code['Total Demand']
        populated_data = []

        for _, period_row in monthly_cashflow.iterrows():
            period = period_row['Period']
            target_profit = period_row['Profit']
            accumulated_profit = 0

            while accumulated_profit < target_profit and clinic_item_code['Remaining Demand'].sum() > 0:
                available_items = clinic_item_code[clinic_item_code['Remaining Demand'] > 0]
                sampled_row = available_items.sample(n=1).iloc[0]
                sampled_revenue = sampled_row['price AUD']
                sampled_expense = sampled_row['cost_material AUD']
                sampled_profit = sampled_revenue - sampled_expense

                clinic_item_code.loc[sampled_row.name, 'Remaining Demand'] -= 1
                populated_data.append({
                    'Period': period,
                    'Code': sampled_row['Code'],
                    'Revenue': sampled_revenue,
                    'Expense': sampled_expense,
                    'Profit': sampled_profit
                })

                accumulated_profit += sampled_profit

        # Handle remaining items in the last period
        last_period = monthly_cashflow['Period'].max()
        remaining_items = clinic_item_code[clinic_item_code['Remaining Demand'] > 0]
        for _, remaining_row in remaining_items.iterrows():
            while remaining_row['Remaining Demand'] > 0:
                remaining_row['Remaining Demand'] -= 1
                populated_data.append({
                    'Period': last_period,
                    'Code': remaining_row['Code'],
                    'Revenue': remaining_row['price AUD'],
                    'Expense': remaining_row['cost_material AUD'],
                    'Profit': remaining_row['price AUD'] - remaining_row['cost_material AUD']
                })

        return pd.DataFrame(populated_data)

    populated_item_codes = populate_item_codes_by_period(merged_item_code, monthly_cashflow)

    # Step 4: Convert numerical periods into calendar dates
    def convert_period_to_date_with_calendar_logic(populated_item_codes):
        start_year = 2024

        def random_date_from_period(period):
            year = start_year + (period - 1) // 12
            month = (period - 1) % 12 + 1
            days_in_month = calendar.monthrange(year, month)[1]
            random_day = random.randint(1, days_in_month)
            return date(year, month, random_day)

        populated_item_codes['Period'] = populated_item_codes['Period'].astype(int)
        populated_item_codes['Date'] = populated_item_codes['Period'].apply(random_date_from_period)
        populated_item_codes['Period'] = populated_item_codes['Date']
        populated_item_codes.drop(columns=['Date'], inplace=True)
        
        return populated_item_codes

    populated_item_codes = convert_period_to_date_with_calendar_logic(populated_item_codes)

    return populated_item_codes


# Generate the complete cashflow with resolved logic
complete_cashflow_df_final_debug = generate_complete_cashflow_final_debug(
    clinic_item_code, cleaned_item_code, forecast_period=12, target_cov=0.15
)

# import ace_tools as tools; tools.display_dataframe_to_user(name="Final Debugged Complete Cashflow DataFrame", dataframe=complete_cashflow_df_final_debug)


Columns in merged_item_code: Index(['Code', 'Total Demand', 'Total Revenue', 'Total Material Cost',
       'Total Duration', 'Medical Officer', 'Total Salary', 'price AUD_x',
       'cost_material AUD_x', 'item_number', 'price AUD_y',
       'cost_material AUD_y'],
      dtype='object')


In [42]:
complete_cashflow_df_final_debug

,Period,Code,Revenue,Expense,Profit,Date
0,1,631,331.0,39.61,291.39,2024-01-05
1,1,833,113.0,7.71,105.29,2024-01-07
2,1,522,133.0,21.00,112.00,2024-01-05
3,1,019,12.0,1.16,10.84,2024-01-04
4,1,632,338.0,40.29,297.71,2024-01-15
...,...,...,...,...,...,...
15431,12,022,6.0,0.67,5.33,2024-12-30
15432,12,022,6.0,0.67,5.33,2024-12-08
15433,12,022,6.0,0.67,5.33,2024-12-14
15434,12,022,6.0,0.67,5.33,2024-12-08


In [11]:
import pandas as pd
import numpy as np
import random
from datetime import date
import calendar

def generate_monthly_cashflow_given_item_code(clinic_item_code, cleaned_item_code, forecast_period, target_cov):
    """
    Optimized generation of a monthly cashflow DataFrame with controlled CoV for profits.
    """
    
    clinic_item_code['price AUD'] = cleaned_item_code['price AUD']
    clinic_item_code['cost_material AUD'] = cleaned_item_code['cost_material AUD']
    
    # Calculate total revenue and expense
    total_revenue = (clinic_item_code['Total Demand'] * clinic_item_code['price AUD']).sum()
    total_expense = (clinic_item_code['Total Demand'] * clinic_item_code['cost_material AUD']).sum()
    total_profit = total_revenue - total_expense

    # Initialize monthly cashflow
    monthly_cashflow = pd.DataFrame({
        'Period': list(range(1, forecast_period + 1)),
        'Revenue': np.zeros(forecast_period),
        'Expense': np.zeros(forecast_period),
        'Profit': np.zeros(forecast_period)
    })

    # Generate Revenue and Expense with controlled CoV for Profit
    mean_revenue = total_revenue / forecast_period
    mean_expense = total_expense / forecast_period
    mean_profit = mean_revenue - mean_expense

    std_profit = target_cov * mean_profit

    # Generate profit distribution with target CoV
    profit_dist = np.random.normal(loc=mean_profit, scale=std_profit, size=forecast_period)
    profit_dist = np.clip(profit_dist, 0, None)  # Ensure no negative profits

    # Adjust Revenue and Expense to match total and profit distribution
    revenue_dist = profit_dist + mean_expense
    expense_dist = revenue_dist - profit_dist

    # Scale Revenue and Expense to match totals
    revenue_dist *= total_revenue / revenue_dist.sum()
    expense_dist *= total_expense / expense_dist.sum()

    # Populate monthly cashflow
    monthly_cashflow['Revenue'] = revenue_dist
    monthly_cashflow['Expense'] = expense_dist
    monthly_cashflow['Profit'] = monthly_cashflow['Revenue'] - monthly_cashflow['Expense']
    
    ### Populate Code
    
    def populate_item_codes_by_period(clinic_item_code, monthly_cashflow):
        """
        Populate item codes into periods based on target profits in the monthly cashflow.
        """
        # Initialize a DataFrame for the final output
        populated_data = []

        # Create a working copy of clinic_item_code with a Remaining Demand column
        clinic_item_code = clinic_item_code.copy()
        clinic_item_code['Remaining Demand'] = clinic_item_code['Total Demand']

        for _, period_row in monthly_cashflow.iterrows():
            period = period_row['Period']
            target_profit = period_row['Profit']
            accumulated_profit = 0

            while accumulated_profit < target_profit and clinic_item_code['Remaining Demand'].sum() > 0:
                # Randomly select an item code
                available_items = clinic_item_code[clinic_item_code['Remaining Demand'] > 0]
                sampled_row = available_items.sample(n=1).iloc[0]

                # Calculate the sampled profit
                sampled_revenue = sampled_row['price AUD']
                sampled_expense = sampled_row['cost_material AUD']
                sampled_profit = sampled_revenue - sampled_expense

                # Deduct one unit of demand from the sampled item code
                clinic_item_code.loc[sampled_row.name, 'Remaining Demand'] -= 1

                # Append to the populated data
                populated_data.append({
                    'Period': period,
                    'Code': sampled_row['Code'],
                    'Revenue': sampled_revenue,
                    'Expense': sampled_expense,
                    'Profit': sampled_profit
                })

                accumulated_profit += sampled_profit

        # Handle remaining item codes in the last period
        last_period = monthly_cashflow['Period'].max()
        remaining_items = clinic_item_code[clinic_item_code['Remaining Demand'] > 0]

        for _, remaining_row in remaining_items.iterrows():
            while remaining_row['Remaining Demand'] > 0:
                remaining_row['Remaining Demand'] -= 1
                populated_data.append({
                    'Period': last_period,
                    'Code': remaining_row['Code'],
                    'Revenue': remaining_row['price AUD'],
                    'Expense': remaining_row['cost_material AUD'],
                    'Profit': remaining_row['price AUD'] - remaining_row['cost_material AUD']
                })

        # Convert to DataFrame
        populated_df = pd.DataFrame(populated_data)

        return populated_df
    

    
    populated_item_codes_df = populate_item_codes_by_period(clinic_item_code, monthly_cashflow)

    def convert_period_to_date_with_calendar_logic(monthly_cashflow_df):
        """
        Convert numerical periods into specific dates based on a calendar starting in January 2024.
        The function assigns a random date within the corresponding month.

        Parameters:
        - monthly_cashflow_df: DataFrame with a Period column.

        Returns:
        - DataFrame with an added Date column.
        """
        # Starting year
        start_year = 2024

        # Function to calculate the specific date for a given period
        def random_date_from_period(period):
            # Calculate the year and month based on the period
            year = start_year + (period - 1) // 12
            month = (period - 1) % 12 + 1

            # Get the number of days in the calculated month and year
            days_in_month = calendar.monthrange(year, month)[1]

            # Generate a random day within the month
            random_day = random.randint(1, days_in_month)

            return date(year, month, random_day)

        # Apply the function to the Period column
        monthly_cashflow_df['Period'] = monthly_cashflow_df['Period'].astype(int)
        monthly_cashflow_df['Date'] = monthly_cashflow_df['Period'].apply(random_date_from_period)
        monthly_cashflow_df['Period'] = monthly_cashflow_df['Date']
        
        monthly_cashflow_df.drop(columns=['Date'], inplace=True)

        return monthly_cashflow_df


    # Convert Period to actual dates starting from January 2024
    converted_cashflow_df = convert_period_to_date_with_calendar_logic(populated_item_codes_df)


    return converted_cashflow_df

In [12]:
generate_monthly_cashflow_given_item_code(clinic_item_code, cleaned_item_code, 12, 0.15)

,Period,Code,Revenue,Expense,Profit
0,2024-01-07,73,22.0,2.70,19.30
1,2024-01-21,927,179.0,23.14,155.86
2,2024-01-27,414,0.0,0.00,0.00
3,2024-01-01,632,84.0,9.57,74.43
4,2024-01-05,15,83.0,12.95,70.05
...,...,...,...,...,...
15431,2024-12-03,22,17.0,1.62,15.38
15432,2024-12-19,22,17.0,1.62,15.38
15433,2024-12-20,22,17.0,1.62,15.38
15434,2024-12-11,22,17.0,1.62,15.38


In [1]:
import pandas as pd
import numpy as np

data = pd.read_csv('clinic_item_code_cashflow.csv')
data

,Period,Code,Revenue,Expense,Profit
0,2024-01-01,22,17.0,1.62,15.38
1,2024-01-01,61,6.0,0.67,5.33
2,2024-01-01,221,0.0,0.00,0.00
3,2024-01-01,986,297.0,33.28,263.72
4,2024-01-01,213,0.0,0.00,0.00
...,...,...,...,...,...
15431,2024-12-31,22,17.0,1.62,15.38
15432,2024-12-31,22,17.0,1.62,15.38
15433,2024-12-31,114,27.0,3.91,23.09
15434,2024-12-31,22,17.0,1.62,15.38


In [2]:
# retrieve month from the date and store as column 'Month'
data['Month'] = pd.to_datetime(data['Period']).dt.strftime('%Y-%m')

#group by month and calculate the sum of revenue, expense and profit
monthly_cashflow = data.groupby('Month').agg(
    Revenue=('Revenue', 'sum'),
    Expense=('Expense', 'sum'),
    Profit=('Profit', 'sum')
).reset_index()

#calculate the CoV of revenue
def calculate_cov(values):
    mean = np.mean(values)
    std_dev = np.std(values)
    return std_dev / mean if mean != 0 else 0

# Calculate CoV of revenue
cov_revenue = calculate_cov(monthly_cashflow['Revenue'])
cov_revenue

0.14482620809713095

In [6]:
import random
from datetime import timedelta, datetime

def add_hourly_period(converted_cashflow_df):
    """
    Add a new column 'Hourly_Period' to the cashflow DataFrame by assigning random times
    to each date in the 'Period' column. The times are in half-hour increments and range
    from 8:00 AM to 12:00 PM and 1:00 PM to 4:00 PM.

    Parameters:
    - converted_cashflow_df: DataFrame with a 'Period' column containing date values.

    Returns:
    - DataFrame with an added 'Hourly_Period' column.
    """
    
    converted_cashflow_df['Period'] = pd.to_datetime(converted_cashflow_df['Period'])
    
    
    # Define valid time slots
    morning_slots = [timedelta(hours=8 + i // 2, minutes=(i % 2) * 30) for i in range(8)]
    afternoon_slots = [timedelta(hours=13 + i // 2, minutes=(i % 2) * 30) for i in range(8)]
    valid_time_slots = morning_slots + afternoon_slots

    # Function to generate a random hourly period
    def generate_random_hourly_period(period_date):
        random_slot = random.choice(valid_time_slots)
        return datetime.combine(period_date, datetime.min.time()) + random_slot

    # Apply the function to generate 'Hourly_Period'
    converted_cashflow_df['Hourly_Period'] = converted_cashflow_df['Period'].apply(generate_random_hourly_period)
    
    # sort by 'Hourly_Period'
    converted_cashflow_df = converted_cashflow_df.sort_values(by='Hourly_Period').reset_index(drop=True)
                                                                                              

    return converted_cashflow_df


# Apply the function to add the 'Hourly_Period' column
converted_cashflow_df_with_hourly = add_hourly_period(data)
converted_cashflow_df_with_hourly

,Period,Code,Revenue,Expense,Profit,Month,Hourly_Period
0,2024-01-01,531,80.0,5.17,74.83,2024-01,2024-01-01 08:00:00
1,2024-01-01,535,66.0,7.20,58.80,2024-01,2024-01-01 08:00:00
2,2024-01-01,72,43.0,6.18,36.82,2024-01,2024-01-01 08:00:00
3,2024-01-01,213,0.0,0.00,0.00,2024-01,2024-01-01 08:30:00
4,2024-01-01,61,6.0,0.67,5.33,2024-01,2024-01-01 08:30:00
...,...,...,...,...,...,...,...
15431,2024-12-31,22,17.0,1.62,15.38,2024-12,2024-12-31 16:30:00
15432,2024-12-31,121,0.0,0.00,0.00,2024-12,2024-12-31 16:30:00
15433,2024-12-31,22,17.0,1.62,15.38,2024-12,2024-12-31 16:30:00
15434,2024-12-31,121,0.0,0.00,0.00,2024-12,2024-12-31 16:30:00


In [3]:
import pickle

# open pickle file and laod as dictionary
with open(r'pkl_files/dataset_1/pool_clinic_df.pkl', 'rb') as f:
    pool_clinic_df = pickle.load(f)
    
    
    

In [4]:
# get all keys in pool_clinic_df as list 
keys = list(pool_clinic_df.keys())
pool_clinic_df['N21701']['Gross Profit']

,Period,Code,Revenue,Expense,Profit,Provider,Year,Quarter,Hourly_Period,Patient ID,Patient Age
0,2022-01-01,416,95.0,26.32608,68.67392,OHT 1,2022,Q1,2022-01-01 10:00:00,Patient 665,15.0
1,2022-01-01,743,112.0,39.87216,72.12784,Dentist 1,2022,Q1,2022-01-01 13:00:00,Patient 361,30.0
2,2022-01-01,161,24.0,17.71608,6.28392,Dentist 1,2022,Q1,2022-01-01 08:30:00,Patient 515,10.0
3,2022-01-01,525,302.0,64.65824,237.34176,Dentist 1,2022,Q1,2022-01-01 11:00:00,Patient 597,43.0
4,2022-01-01,415,123.0,45.17216,77.82784,OHT 1,2022,Q1,2022-01-01 13:00:00,Patient 598,19.0
...,...,...,...,...,...,...,...,...,...,...,...
36592,2024-12-31,022,20.0,15.85608,4.14392,OHT 1,2024,Q12,2024-12-31 08:30:00,Patient 507,58.0
36593,2024-12-31,022,20.0,15.85608,4.14392,Dentist 1,2024,Q12,2024-12-31 09:00:00,Patient 674,36.0
36594,2024-12-31,022,20.0,15.85608,4.14392,OHT 1,2024,Q12,2024-12-31 13:00:00,Patient 226,23.0
36595,2024-12-31,022,20.0,15.85608,4.14392,OHT 1,2024,Q12,2024-12-31 11:30:00,Patient 753,41.0


In [5]:
pool_clinic_df['N21701']['Indirect Cost'].head()

,Period,Revenue,Expense,Hourly_Period,Profit,Year,Quarter
0,2022-01-31,0,27448.0,2022-01-31 09:30:00,-27448.0,2022,Q1
1,2022-02-28,0,27448.0,2022-02-28 16:30:00,-27448.0,2022,Q1
2,2022-03-31,0,27448.0,2022-03-31 15:30:00,-27448.0,2022,Q1
3,2022-04-30,0,27448.0,2022-04-30 14:30:00,-27448.0,2022,Q2
4,2022-05-31,0,27448.0,2022-05-31 13:30:00,-27448.0,2022,Q2


In [ ]:
class ModelTransformData:
    def __init__(self, transaction_data, indirect_expense_data):
        self.transaction_data = transaction_data
        self.indirect_expense_data = indirect_expense_data
        
        # create empty excel file
        self.company_variables = pd.ExcelWriter('company_variables.xlsx', engine='xlsxwriter')

    def generate_main_variables(self):
        
        # calculate average net sales per year based on revenue
        net_sales_yearly = self.transaction_data.groupby('Year')['Revenue'].sum().reset_index()
        average_net_sales = net_sales_yearly['Revenue'].mean()
        
        # calculate cogs per year based on two columns of Expense
        cogs_yearly = self.transaction_data.groupby('Year')['Expense'].sum().reset_index()
        average_cogs = cogs_yearly['Expense'].mean()
        
        # calculate indirect expense from indirect expense data per year 
        indirect_expense_yearly = self.indirect_expense_data.groupby('Year')['Expense'].sum().reset_index()
        average_indirect_expense = indirect_expense_yearly['Expense'].mean()
        
        # year-on-year growth rate for net sales
        yearly_net_sales = self.transaction_data.groupby('Year')['Revenue'].sum().reset_index()
        yearly_net_sales['Net Sales Growth'] = yearly_net_sales['Revenue'].pct_change()
        average_yearly_net_sales_growth = yearly_net_sales['Net Sales Growth'].mean()
        
        # calculate relative variation of net sales in monthly basis or the coefficient of variation (CoV)
        
        self.transaction_data['Period'] = pd.to_datetime(self.transaction_data['Period'])
        monthly_net_sales = self.transaction_data.groupby([self.transaction_data['Year'], self.transaction_data['Period'].dt.month])['Revenue'].sum().reset_index()
        cov_monthly_net_sales = monthly_net_sales['Revenue'].std() / monthly_net_sales['Revenue'].mean()
        
        # calculate number of active patients (select unique patients for the 2 last most recent year)
        number_active_patients = self.transaction_data[self.transaction_data['Year'] >= yearly_net_sales['Year'].max() - 1]['Patient ID'].nunique()
        
        
        # calculate relative variation of patient spending for entire period
        patient_spending = self.transaction_data.groupby('Patient ID')['Revenue'].sum().reset_index()
        cov_patient_spending = patient_spending['Revenue'].std() / patient_spending['Revenue'].mean()
        

        
        
        
        return average_net_sales, average_cogs, average_indirect_expense, average_yearly_net_sales_growth, cov_monthly_net_sales, number_active_patients, cov_patient_spending
    
    def store_main_variables_to_excel(self):
        # Generate main variables
        average_net_sales, average_cogs, average_indirect_expense, average_yearly_net_sales_growth = self.generate_main_variables()
        
        # Create a DataFrame with the desired layout
        main_variables = pd.DataFrame({
            'Variable': ['Net Sales', 'COGS', 'Other Expense', 'Net Sales Growth', 'Number of Active Patients', 'Relative Variation of Patient Spending'],
            'Value': [average_net_sales, -average_cogs, -average_indirect_expense, average_yearly_net_sales_growth, number_active_patients, cov_patient_spending]
        })
        
        # Store the main variables in the Excel file
        main_variables.to_excel(self.company_variables, sheet_name='main_variables', index=False)
        
        # Save and close the Excel writer
        # self.company_variables.close()
        
        # return main_variables
    
    def store_monthly_net_sales_to_excel(self):
        # Calculate monthly net sales
        
        self.transaction_data['Period'] = pd.to_datetime(self.transaction_data['Period'])
        
        monthly_net_sales = self.transaction_data.groupby([self.transaction_data['Year'], self.transaction_data['Period'].dt.month])['Revenue'].sum().reset_index()
        
        monthly_net_sales = monthly_net_sales.rename(columns={'Period': 'Month', 'Revenue': 'Net Sales'})
        
        
        # Store the monthly net sales in the Excel file
        monthly_net_sales.to_excel(self.company_variables, sheet_name='monthly_net_sales', index=False)
        
        # Save and close the Excel writer
        # self.company_variables.close()
        
        # return monthly_net_sales
    
    def store_yearly_net_sales_to_excel(self):
        # Calculate yearly net sales
        
        yearly_net_sales = self.transaction_data.groupby('Year')['Revenue'].sum().reset_index()
        
        yearly_net_sales = yearly_net_sales.rename(columns={'Revenue': 'Net Sales'})
        
        # Store the yearly net sales in the Excel file
        yearly_net_sales.to_excel(self.company_variables, sheet_name='yearly_net_sales', index=False)
        
        # Save and close the Excel writer
        # self.company_variables.close()
        
        # return yearly_net_sales
    
    def store_dentist_contribution_to_excel(self):
        
        # groupby column provider and calculate the sum of revenue
        dentist_contribution = self.transaction_data.groupby('Provider')['Revenue'].sum().reset_index()
        dentist_contribution = dentist_contribution.rename(columns={'Revenue': 'Sales Contribution ($)', 'Provider': 'Dentist Provider'})
        dentist_contribution['Possibly Leaving?'] = False
        
        # Store the dentist contribution in the Excel file
        dentist_contribution.to_excel(self.company_variables, sheet_name='dentist_contribution', index=False)
        
        # return dentist_contribution
    
    def store_patient_transaction_to_excel(self):
        
        # groupby column patient and calculate the sum of revenue
        patient_transaction = self.transaction_data
    
        # Store the patient transaction in the Excel file
        patient_transaction.to_excel(self.company_variables, sheet_name='patient_transaction', index=False)
        
        # return patient_transaction
    
    def prepare_all_data(self):

        self.store_main_variables_to_excel()
        self.store_monthly_net_sales_to_excel()
        self.store_yearly_net_sales_to_excel()
        self.store_dentist_contribution_to_excel()
        self.store_patient_transaction_to_excel()
        
        self.company_variables.close()
        
        return self.company_variables
    
    
    
        

In [8]:
import pandas as pd
import numpy as np

model = ModelTransformData(pool_clinic_df['N21701']['Gross Profit'], pool_clinic_df['N21701']['Indirect Cost'])
model.generate_main_variables()
model.store_main_variables_to_excel()

,Variable,Value
0,Net Sales,996902.000000
1,COGS,-374048.330555
2,Other Expense,-329376.000000


In [9]:
monthly = model.store_monthly_net_sales_to_excel()
monthly.head()

,Year,Month,Net Sales
0,2022,1,89370.0
1,2022,2,100113.0
2,2022,3,96571.0
3,2022,4,88182.0
4,2022,5,62365.0


In [10]:
model.store_yearly_net_sales_to_excel()

,Year,Net Sales
0,2022,1016926.0
1,2023,980736.0
2,2024,993044.0


In [11]:
model.store_dentist_contribution_to_excel()

,Dentist Provider,Sales Contribution ($),Possibly Leaving?
0,Dentist 1,1492176.0,False
1,OHT 1,1283727.0,False
2,Specialist 1,214803.0,False


In [ ]:
yearly_net_sales = pool_clinic_df['N21701']['Gross Profit'].groupby('Year')['Revenue'].sum().reset_index()
yearly_net_sales['Net Sales Growth'] = yearly_net_sales['Revenue'].pct_change()


In [14]:
yearly_net_sales['Net Sales Growth'].mean()

-0.011518942498334794

In [18]:
# pool_clinic_df['N21701']['Gross Profit']['Period'] = pd.to_datetime(pool_clinic_df['N21701']['Gross Profit']['Period'])
monthly_net_sales = pool_clinic_df['N21701']['Gross Profit'].groupby([pool_clinic_df['N21701']['Gross Profit']['Year'], pool_clinic_df['N21701']['Gross Profit']['Period'].dt.month])['Revenue'].sum().reset_index()
monthly_net_sales['Revenue'].std() / monthly_net_sales['Revenue'].mean()

0.13876311725096208

In [ ]:
import pandas as pd 
import numpy as np
import pickle 


data = 